# União das tabelas de Operações e de Acidentes

### Importação dos requisitos

In [ ]:
import pandas as pd
from resolve_path import ajuste_path

### Leitura das tabelas necessárias

In [ ]:
pathUtil = ajuste_path('data/util/')
pathInput = ajuste_path('data/input/')

df_acid = pd.read_csv(
    pathUtil + 'acidentes/acidentes_preparado.csv', encoding='utf-8', sep='#')

df_op = pd.read_csv(
    pathUtil + 'os/operacoes_preparado.csv', encoding='utf-8')

### Guardando o nome das colunas para uso posterior

In [ ]:
colunas_os = list(df_op.columns)
colunas_acid = df_acid.columns

### Traduz o tipo das colunas lidas

In [ ]:
df_acid['data'] = pd.to_datetime(df_acid['data'])
df_acid['id'] = df_acid['id'].astype(int)

df_op['data_inicio'] = pd.to_datetime(df_op['data_inicio'])
df_op['data_fim'] = pd.to_datetime(df_op['data_fim'])
# necessário adicionar um dia pois a data de operação não inclui a hora
# enquanto o acidente sim
df_op['data_fim'] = df_op['data_fim'] + pd.Timedelta(days=1)
df_op.dtypes

### Associação dos acidentes com as operações que estavam ativas no dia do acidente

In [ ]:
df_op['merge'] = False

colunas_os.append('merge')

# cria um set que não permite repetição para
# guardar quais foram as colunas já adicionadas
colunas_acidente_na_op = set()

# adiciona cada acidente a todas as operações
# ativas daquele funcionário,
# com a intenção de depois filtrar
# qual a operação correta
for idx, row in df_acid.iterrows():

    df_op.loc[
        ((df_op['data_inicio'] <= row['data'])
         & (df_op['data_fim'] > row['data']))  # mesma data
        & (df_op['no_pessoal'] == row['id']),  # mesma matricula
        'merge'
    ] = True  # assim guarda quais são as linhas a serem associadas

    if df_op[df_op['merge']].shape[0] == 0:
        continue  # se não há operações, pula o acidente

    if 'id_acidente' not in colunas_acidente_na_op:
        # primeira adição de colunas
        for key in row.index:
            colunas_acidente_na_op.add(key + '_acidente')
            df_op.loc[df_op['merge'], key + '_acidente'] = row[key]

        df_op[list(colunas_acidente_na_op)] = df_op[list(
            colunas_acidente_na_op)].astype(object)
        continue

    if df_op.loc[df_op['merge'], 'id_acidente'].isna().all():
        # caso onde todas as operações ativas não causaram nenhum acidente
        for key in row.index:
            colunas_acidente_na_op.add(key + '_acidente')
            df_op.loc[df_op['merge'], key + '_acidente'] = row[key]

    else:
        # caso onde alguma operação já causou algum outro acidente
        # adiciona-se ao final a operação duplicada com o novo
        # acidente associado, com a intenção de que isto será
        # limpado no momento de filtrar as associações corretas
        df_duplicatas = df_op.loc[df_op['merge'], colunas_os].reset_index()
        lim_sup = len(df_op)
        df_duplicatas.index = df_duplicatas.index + lim_sup + 1
        df_op = pd.concat([df_op, df_duplicatas], axis=0)
        df_op.loc[:lim_sup, 'merge'] = False

        for key in row.index:
            colunas_acidente_na_op.add(key + '_acidente')
            df_op.loc[df_op['merge'], key + '_acidente'] = row[key]

    df_op['merge'] = False  # limpa as operações para a nova iteração

### Limpa o resultado da associação

In [ ]:
col_float = ['potencial_acidente',
             'latitude_acidente',
             'longitude_acidente',
             'id_acidente',
             ]

col_datetime = ['data_acidente']

df_op[col_float] = df_op[col_float].astype(float)

for col in col_datetime:
    df_op[col] = pd.to_datetime(df_op[col])

df_op.drop(columns=['merge', 'index'], axis=1, inplace=True)

### Lê o arquivo onde foi guardada a associação escolhida à mão 
Análise compreensiva feita a partir da associação inicial apenas através da data

Foram analisados diversos fatores para a associação como a descrição do acidente,
o txt. breve da operação, o local de instalação, a coordenada do acidente, a duração
da operação, a proximidade de datas, a quantidade de trabalho entre outros ainda.


In [ ]:
df_associacao = pd.read_csv(
    pathInput + "associa_acidentes_operacao_coordenadas_confianca.csv")

In [ ]:
df_associacao.info()

### Preprocessa a coluna confiança
A coluna 'Confianca' representa a associação escolhida à mão, para os casos positivos. O valor indica parcialmente a confiança da escolha por aquela ser a operação que causou o acidente

In [ ]:
df_associacao['Confianca'] = df_associacao['Confianca'].apply(
    lambda x: float(x.replace(',', '.')) if isinstance(x, str) else x)

df_associacao['Confianca'].value_counts()

### Adiciona ao DataFrame as informações da associação à mão
usa-se de sufixo '_correto' para indicar as informações provenientes da associação à mão (escolhido por ser a associação mais "correta" do que a feita automaticamente)

In [ ]:
df_associacao = df_associacao[df_associacao['Confianca'] > 0]

# guardar os nomes das colunas da associacao a mao univocamente
colunas_correto = set()

# realiza a adição
for idx, row in df_associacao.iterrows():
    # informacoes a serem usadas para achar qual foi a associacao escolhida
    ordem = row['Ordem']
    op = row['Operação']
    id = row['ID']
    data = row['Dt']

    mask = ((df_op['ordem'] == ordem)
            & (df_op['operacao'] == op)
            & (df_op['no_pessoal'] == id)
            & ((df_op['data_inicio'] <= data)
               & (df_op['data_fim'] > data))
            )

    ops_ativas = df_op.loc[mask]
    if ops_ativas.shape[0] > 1:
        # caso onde houve mais de uma confirmação de uma operação com um mesmo funcionário
        # se escolhe o lançamento mais recente
        ultimo_lancamento = ops_ativas['contador'].max()
        mask = (df_op['contador'] == ultimo_lancamento) & mask
    for key in row.index:
        colunas_correto.add(key.lower() + '_correto')
        df_op.loc[mask, key.lower() + '_correto'] = row[key]

### Fazer testes para certificar onde e como '_correto' diferencia do '_acidente'

In [ ]:
df_teste = df_op[df_op['id_correto'].notna() | df_op['id_acidente'].notna()]

df_acidentes_possivel_errados = df_teste[df_teste['id_correto'].isna(
) & df_teste['id_acidente'].notna()]

### Corrigindo a associação a partir do link feito à mão
Despopula-se os campos onde estava uma associação não 'correta'.

In [ ]:
df_op.loc[df_op['id_correto'].isna(),
          list(colunas_acidente_na_op)] = pd.NA

### Limpeza das operações duplicadas, deixando cada uma com um acidente
Limpeza feita através da descrição pois para estes casos existem as linhas duplicadas de duas operações as duas operações associadas em cada linha a um dos dois acidentes, após a operação se tem cada operação com um acidente, conforme o exemplo:

operação1 <=> acidente1

operação2 <=> acidente1

operação1 <=> acidente2

operação2 <=> acidente2

se torna 

operação1 <=> acidente1

operação2 <=> acidente2


In [ ]:
# já foram usadas as outras informações que vieram do _correto,
# mas se viessem mais informações poderia não ser utilizada a
# descrição para discriminar qual foi o verdadeiro acidente causado.
# Recomenda-se em futuras associações à mão que se inclua mais colunas
# que de preferência tenha valores únicos.
df_op['desc_acid'] = df_op['descricao_acidente'].str.strip().str[:20]
df_op['desc_corr'] = df_op['descrição_correto'].str.strip().str[:20]
df_op['diferente'] = False
df_op.loc[df_op['desc_acid'] != df_op['desc_corr'], 'diferente'] = True
df_op.loc[df_op['id_acidente'].isna(), 'diferente'] = False
df_op.fillna({'diferente': False}, inplace=True)

df_op = df_op[~df_op['diferente']]

cols_intermediarias = ['desc_acid', 'desc_corr', 'diferente']
df_op.drop(columns=cols_intermediarias, inplace=True)

#### DataFrame para fazer a comparação entre a associação automática e a feita à mão

In [ ]:
df_teste = df_op[df_op['id_correto'].notna() | df_op['id_acidente'].notna()]

### Remove as colunas adicionadas puramente para a associação

In [ ]:
df_op.drop(columns=colunas_correto, inplace=True)

In [ ]:
df_op.info()

In [ ]:
df_op.to_csv(pathUtil + 'dataset_associado.csv', encoding='utf-8', index=False)